In [ ]:
## setup analysis
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
from analysis.lib.purification import purify_pq as ppq; reload(ppq)
from analysis.lib.purification import purify_analysis as pa; reload(pa)
%matplotlib inline

theta = 'pi/4'

folder_lt3,folder_lt4 = pa.get_folders_from_theta(theta)
ssro_calib_lt3,ssro_calib_lt4 = pa.get_ssro_calibs()

Purify = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)

In [ ]:
Purify.get_tstamps_and_offsets(contains = 'Purify', verbose = False)

In [ ]:
Purify.calculate_sequence_time(max_w2 = 500)

In [ ]:
thetas =['pi/4','pi/5','pi/6','pi/8']

num_first_clicks = np.zeros(len(thetas))
total_syncs = np.zeros(len(thetas))

for i, theta in enumerate(thetas):
    
    folder_lt3,folder_lt4 = pa.get_folders_from_theta(theta)
    ssro_calib_lt3,ssro_calib_lt4 = pa.get_ssro_calibs()
    Purify = pa.purify_analysis('purification_analysis',folder_lt3,folder_lt4,ssro_calib_lt3,ssro_calib_lt4)
    Purify.get_tstamps_and_offsets(contains = 'Purify', verbose = False)
    num_first_clicks[i], total_syncs[i] = Purify.calculate_sequence_time(print_details = False, return_click_prob = True)

In [ ]:
realpopvals = np.array([0.49214634,  0.39799117,  0.27649945,  0.1760004])

probs = num_first_clicks/total_syncs

fit_result = fit.fit1d(realpopvals, probs,common.fit_line,0,1,fixed = [0])
plot.plot_fit1d(fit_result)

In [ ]:
b = fit_result['params_dict']['b']
error = fit_result['error_dict']['b']
print b, error